In [187]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
from random import seed

In [188]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

In [189]:
# Scenario parameters
a_si = [[1,1], [1,0], [0,1]] # don't include [0,0]
b_sj = [[1,1,1], [1,0,1], [1,0,0], [1,0,1], [1,1,0], [0,1,0], [0,0,1]] # don't include [0,0,0]

In [190]:
Scenarios = [[x,y] for x in a_si for y in b_sj]

In [191]:
Scenarios

[[[1, 1], [1, 1, 1]],
 [[1, 1], [1, 0, 1]],
 [[1, 1], [1, 0, 0]],
 [[1, 1], [1, 0, 1]],
 [[1, 1], [1, 1, 0]],
 [[1, 1], [0, 1, 0]],
 [[1, 1], [0, 0, 1]],
 [[1, 0], [1, 1, 1]],
 [[1, 0], [1, 0, 1]],
 [[1, 0], [1, 0, 0]],
 [[1, 0], [1, 0, 1]],
 [[1, 0], [1, 1, 0]],
 [[1, 0], [0, 1, 0]],
 [[1, 0], [0, 0, 1]],
 [[0, 1], [1, 1, 1]],
 [[0, 1], [1, 0, 1]],
 [[0, 1], [1, 0, 0]],
 [[0, 1], [1, 0, 1]],
 [[0, 1], [1, 1, 0]],
 [[0, 1], [0, 1, 0]],
 [[0, 1], [0, 0, 1]]]

In [192]:
num_Scenarios = len(Scenarios)
p_scen = 1/num_Scenarios

In [193]:
# Product Demand
demand = np.random.randint(0,50,(num_Scenarios, Products,Market))

In [194]:
# Cost of opening
f_i = [200, 50]
f_j = [75, 100, 50]

In [195]:
# Unit cost of manufacturing product 
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [196]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,5, (Products, Manufacturing_plants, Distribution))

In [197]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,5, (Products, Distribution, Market))

In [198]:
# Plant Capacities
Capacities_i = np.random.randint(300,500,(Manufacturing_plants)) # in volume !!! (metres cubed)
Capacities_j = np.random.randint(300,500,(Distribution)) # in volume !!! (metres cubed)
Capacities_l = np.zeros((Products,Outsourced)) # in terms of products 
np.fill_diagonal(Capacities_l, random.randint(15,20))

In [199]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
#np.fill_diagonal(Supplier_cost[0], random.uniform(10,15))
for m in range(Products):
    for l in range(Outsourced):
        if m == l:
            Supplier_cost[0][m][l] = random.uniform(10,15)
            Supplier_cost[1][m][l] = random.uniform(15,20)
            
#np.fill_diagonal(Supplier_cost[1], random.uniform(15,20))

In [200]:
Supplier_cost

array([[[12.71593269,  0.        ],
        [ 0.        , 13.36155138]],

       [[18.33600142,  0.        ],
        [ 0.        , 15.14047352]]])

In [201]:
# Cost of transporting product m from outsourced facility l to j
T_O_DC = np.zeros((Products, Outsourced, Distribution))
T_O_DC[0][0] = np.random.uniform(10,15,(Distribution))
T_O_DC[1][1] = np.random.uniform(10,15,(Distribution))

In [202]:
# Cost of shipping product m from outsourced facility l to k
T_O_MZ = np.zeros((Products, Outsourced, Market))
T_O_MZ[0][0] = np.random.uniform(15,20,(Market))
T_O_MZ[1][1] = np.random.uniform(15,20,(Market))

In [203]:
# Product volume 
volume = np.random.randint(2,5,(Products))

In [204]:
# unit cost of lost sales 
lost_sales = np.random.randint(50,100,(Market,Products))

In [205]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

In [206]:
## Model

In [207]:
grbModel = Model('synthetic')

In [208]:
def SetGurobiModel():
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel.addVar(vtype = GRB.BINARY)
    
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products):
                U_km[s,k,m] = grbModel.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[s,m,l] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for i in range(Manufacturing_plants):
                Q_im[s,m,i] = grbModel.addVar(vtype = GRB.INTEGER)
            
    for s in range(num_Scenarios):  
        for m in range(Products):
            for i in range(Manufacturing_plants):
                for j in range(Distribution):
                    Y_ijm[s,m,i,j] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for j in range(Distribution):
                for k in range(Market): 
                    Z_jkm[s,m,j,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for j in range(Distribution):
                    T_ljm[s,m,l,j] = grbModel.addVar(vtype = GRB.INTEGER)      
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for k in range(Market):
                    T_lkm[s,m,l,k] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                y_lm[s,m,l] = grbModel.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj()
    ModelCons()

In [221]:
def SolveModel():
    
    grbModel.params.OutputFlag = 0
    grbModel.optimize()
    
    # get variable values 
    v_val_x_i = grbModel.getAttr('x', x_i)
    v_val_x_j = grbModel.getAttr('x', x_j)
    v_val_U_km = grbModel.getAttr('x', U_km)
    v_val_V_lm = grbModel.getAttr('x', V_lm)
    v_val_Q_im = grbModel.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel.getAttr('x', T_lkm)
    v_val_y_lm = grbModel.getAttr('x', y_lm)
    
    obj = grbModel.getObjective()
    print("obj val: ", obj.getValue())   
    
    Summary_dict["OpenMPs"] = np.sum(v_val_x_i.values())
    Summary_dict["OpenDCs"] = np.sum(v_val_x_j.values())
    
    for s in range(num_Scenarios):        
        Summary_dict["Purchasing_" + str(s)] = sum([v_val_V_lm[(s,m,l)] for m in range(Products) for l in range(Outsourced)])    
        Summary_dict["Production_" + str(s)] = sum([v_val_Q_im[(s,m,i)] for m in range(Products) for i in range(Manufacturing_plants)])
        Summary_dict["LostSales_" + str(s)] = sum([v_val_U_km[(s,k,m)] for m in range(Products) for k in range(Market)])
        Summary_dict["OutsourceToDC_" + str(s)] = sum([v_val_T_ljm[(s,m,l,j)] for m in range(Products) for l in range(Outsourced) for j in range(Distribution)])
        Summary_dict["OutsourceToMarket_" + str(s)] = sum([v_val_T_lkm[(s,m,l,k)] for m in range(Products) for l in range(Outsourced) for k in range(Market)]) 
    
    Cost_dict["Opening"] =  get_opening_costs(v_val_x_i, v_val_x_j)
    #Cost_dict["InHouseShipping"] = get_shipping_costs(v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[0]
    #Cost_dict["OutsourceShipping"] = get_shipping_costs(v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[1]
    #Cost_dict["Production"] = get_production_cost(v_val_Q_im)
    #Cost_dict["Purchasing"] = get_purchase_costs(v_val_V_lm, v_val_y_lm)
    #Cost_dict["LostSales"] = get_lost_cost(v_val_U_km)    
    
    return

In [222]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 
        
    for s in range(num_Scenarios):
        ship_1 = 0
        ship_2 = 0
        ship_3 = 0
        ship_4 = 0
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                for m in range(Products):
                    ship_1 += Transportation_i_j[m][i][j]*Y_ijm[s,m,i,j]

        for j in range(Distribution):
            for k in range(Market):
                for m in range(Products):
                    ship_2 += Transportation_j_k[m][j][k]*Z_jkm[s,m,j,k]

        for l in range(Outsourced):
            for j in range(Distribution):
                for m in range(Products):
                    ship_3 += T_O_DC[m][l][j]*T_ljm[s,m,l,j]

        for l in range(Outsourced):
            for k in range(Market):
                for m in range(Products):
                    ship_4 += T_O_MZ[m][l][k]*T_lkm[s,m,l,k]
                    
        total_shipment += ship_1 + ship_2 + ship_3 + ship_4

        # Production
        pr_cost = 0
        for i in range(Manufacturing_plants):
            for m in range(Products):
                pr_cost += Manufacturing_costs[i][m]*Q_im[s,m,i]
                
        total_pr_cost += pr_cost

        # Buying from outsource cost
        b_cost = 0
        for l in range(Outsourced):
            for m in range(Products):
                b_cost += Supplier_cost[0][m][l]*V_lm[s,m,l] + Supplier_cost[1][m][l]*y_lm[s,m,l]
                
        total_b_cost += b_cost

        #Lost Sales
        l_cost = 0
        for k in range(Market):
            for m in range(Products):
                l_cost += lost_sales[k][m]*U_km[s,k,m]
                
        total_l_cost += l_cost

    grb_expr += OC_1 + OC_2 + p_scen*(total_shipment + total_pr_cost + total_b_cost + total_l_cost)
    
    grbModel.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 


In [223]:
# Model Constraints

def ModelCons():
    
    # Network Flow

    grbModel.addConstrs(quicksum(Y_ijm[s,m,i,j] for j in range(Distribution)) == Q_im[s,m,i]
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants) for m in range(Products))

    grbModel.addConstrs(quicksum(Z_jkm[s,m,j,k] for k in range(Market)) ==
                        (quicksum(Y_ijm[s,m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[s,m,l,j] for l in range(Outsourced))) for s in range(num_Scenarios)
                        for j in range(Distribution) for m in range(Products))

    grbModel.addConstrs((quicksum(Z_jkm[s,m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[s,m,l,k] for l in range(Outsourced))) == (demand[s][m][k] - U_km[s,k,m])
                        for s in range(num_Scenarios) for k in range(Market) for m in range(Products))
    
    # Can only get one product type from each outsourced facility
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                if l != m:
                    grbModel.addConstr(V_lm[s,m,l] == 0)

        for m in range(Products):
            for l in range(Outsourced):
                for j in range(Distribution):
                    if l != m:
                        grbModel.addConstr(T_ljm[s,m,l,j] == 0)

        for m in range(Products):
            for l in range(Outsourced):
                for k in range(Market):
                    if l != m:
                        grbModel.addConstr(T_lkm[s,m,l,k] == 0)
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel.addConstrs(V_lm[s,m,l] == quicksum(T_ljm[s,m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for k in range(Market)) for s in range(num_Scenarios) 
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel.addConstrs(quicksum(volume[m]*Q_im[s,m,i] for m in range(Products)) <= Scenarios[s][0][i]*Capacities_i[i]*x_i[i] 
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants))
    
    grbModel.addConstrs(quicksum(volume[m]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[s,m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[s][1][j]*Capacities_j[j]*x_j[j] for s in range(num_Scenarios) for s in range(num_Scenarios)
                        for j in range(Distribution))
    
    grbModel.addConstrs((V_lm[s,m,l] <= (Capacities_l[m][l])) for s in range(num_Scenarios)
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (5 is arbitrary)
    grbModel.addConstrs((V_lm[s,m,l] - (Capacities_l[m][l] - 5)*y_lm[s,m,l])
                         <= 5 for s in range(num_Scenarios) for m in range(Products) for l in range(Outsourced))   
    
    
    return   


In [224]:
def get_opening_costs(x1, x2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    
    return(Opening)
   
def get_shipping_costs(Y, Z, T1, T2):
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)
    
    return(in_house_shipping, outsourced_shipping)

def get_production_cost(Q):

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q[m,i]
            
    return(np.round(pr_cost))

def get_purchase_costs(V, y):    

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V[m,l] + Supplier_cost[1][m][l]*y[m,l]
            
    return(np.round(b_cost))

def get_lost_cost(U):
    
    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U[k,m]
            
    return(np.round(l_cost))

In [225]:
def run_Model():    
        
    SetGurobiModel()
    SolveModel()

In [226]:
def dict_to_dataframe(Dict):
    return pd.DataFrame([list(Dict.values())], columns = list(Dict.keys()))

In [227]:
run_Model() # No disrupted facilities

obj val:  1745.1571149810927


In [228]:
dict_to_dataframe(Summary_dict)

,OpenMPs,OpenDCs,Purchasing_0,Production_0,LostSales_0,OutsourceToDC_0,OutsourceToMarket_0,Purchasing_1,Production_1,LostSales_1,...,Purchasing_19,Production_19,LostSales_19,OutsourceToDC_19,OutsourceToMarket_19,Purchasing_20,Production_20,LostSales_20,OutsourceToDC_20,OutsourceToMarket_20
0,2.0,3.0,0.0,123.0,0.0,0.0,0.0,0.0,209.0,0.0,...,32.0,162.0,16.0,0.0,32.0,0.0,101.0,0.0,0.0,0.0


In [249]:
correct = 0
for s in range(num_Scenarios):
    if (Summary_dict['Purchasing_'+str(s)] + Summary_dict['Production_' + str(s) ]+ Summary_dict['LostSales_' + str(s)] == 
          np.sum(demand[s])):
        correct += 1
correct == num_Scenarios

True

In [254]:
Purchasing = [Summary_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production = [Summary_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales = [Summary_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
OutsourceToDC = [Summary_dict['OutsourceToDC_' + str(s)] for s in range(num_Scenarios)]
OutsourceToMarket = [Summary_dict['OutsourceToMarket_' + str(s)] for s in range(num_Scenarios)]

In [261]:
Unit_df = pd.DataFrame(list(zip(Purchasing, Production, LostSales, OutsourceToDC, OutsourceToMarket)), 
             columns = ["Purchasing", "Production", "LostSales", "OutsourceToDC", "OutsourceToMarket"])

In [262]:
Unit_df # number of units that the company purchases & ships to DCs or Market zones, produces, & incurs in lost sales

,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,0.0,123.0,0.0,0.0,0.0
1,0.0,209.0,0.0,0.0,0.0
2,0.0,205.0,0.0,0.0,0.0
3,0.0,144.0,0.0,0.0,0.0
4,0.0,213.0,0.0,0.0,0.0
5,32.0,162.0,6.0,0.0,32.0
6,32.0,161.0,2.0,0.0,32.0
7,0.0,112.0,0.0,0.0,0.0
8,23.0,175.0,0.0,23.0,0.0
9,0.0,153.0,0.0,0.0,0.0
